In [ ]:
!pip install transformers -U

In [ ]:
!pip freeze | grep transformers

In [ ]:
!pip uninstall urllib3 -y

In [ ]:
!pip install urllib3==1.26.15

In [2]:
######################################################################################################
# Model Registration

from refractml import *
from refractml.constants import MLModelFlavours

# new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
from transformers import AutoTokenizer
import torch
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        import os
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
        from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
        import torch
        from torch import cuda
        self.model = None
        self.tokenizer = None
#        self.token_path = '/data/models/mistralai_base/token.pkl'
#        self.model_path = '/data/models/mistralai_base/qa.pkl'
        if self.model is None:
            print("Load Model From Data Volume")
            self.model = AutoModelForCausalLM.from_pretrained("/data/huggingface/cache/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670",device_map = 'cpu',torch_dtype=torch.bfloat16,trust_remote_code=True)
#             self.model_loaded =torch.load(self.model_path)
#            self.model.generate(bos_token_id=tokenizer.bos_token_id)
#            print("model loaded ")

        if self.tokenizer is None:
            print("tokenizer object is loading from data section")
            self.tokenizer = AutoTokenizer.from_pretrained("/data/huggingface/cache/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670",trust_remote_code=True)
#             self.tokenizer = torch.load(self.token_path)
            print("tokenizer loaded")

        self.model.generate(bos_token_id=self.tokenizer.bos_token_id)
        print("model loaded ")
#        !sudo chmod 777 /home/mosaic-ai/.cache/huggingface
    
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[Input data, ..]. It could be:
               A List Mapping of All Value Can Be one of : 
                   - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                   - List[numpy.array(), numpy.array(), ...]
                   - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                   - List[ SingleSample, SingleSample]
                   
        :return: (n_inputs, payload's)
        
        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 
    
    def pre_processing_fn(self,payload):
        # All preprocessing step must occur in this section
        # Takes Single Sample -> Returns Single Sample
        
        # Not Doing Any Preprocessing Hence Returned payload
        print("payload is ", payload)
        
        return payload

    def prediction_fn(self,
                      model: Any,
                      pre_processed_input 
                      ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
#         device = 'cuda' if cuda.is_available() else 'cpu'
#         print(f'Device: ------------- {device}')
        input_data = pre_processed_input
        print(type(input_data))
        print(input_data)
        prompt = input_data
        input_ids=self.tokenizer.encode(prompt,return_tensors='pt')
        output_ids=self.model.generate(input_ids,max_length=500,temperature=0.6,bos_token_id=self.tokenizer.bos_token_id,pad_token_id=self.tokenizer.pad_token_id)
        generated_text=self.tokenizer.decode(output_ids[0],skip_special_tokens=True)
        #print(generated_text)
        #print(sequences[0]['generated_text'])
        #print("inference generated")
        return generated_text

    class Meta:    
        # List of Callables() can be attached For Calling After AnSd Before Scoring
        def __init__(self):
            self.name = "Pre Hooked Me !"
            self.pre_call_hooks.append(self.print_)
        def print_(self):
            print(self.name)
        pre_call_hooks = []
        post_call_hooks = []

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/tmp/pip_packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
prompt = "How to reverse a string in python?"
import requests
req = requests.Request()
req.json = {"payload":prompt
            }

In [4]:
score_ = ScoreTemplateExample()

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp92pnk_b4
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp92pnk_b4/_remote_module_non_scriptable.py


Load Model From Data Volume


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded 


In [ ]:
import time
from datetime import datetime
print("Today's date:", datetime.now())
t1 = time.time()
print(t1)
model_predictions = score_.score(None, req, dry_run=True)
print("Microsoft-phi2 Testing")
t2 = time.time()
t2-t1
print("Today's date:", datetime.now())
print("ended the pipe")
print(t2-t1)

In [ ]:
print(type(model_predictions))
print(len(model_predictions))
print((model_predictions))
print((model_predictions[0]))
print(type(model_predictions[0]))

In [ ]:
register_model(None,
               ScoreTemplateExample,
               "Microsoft_phi2_Testing",
               "Microsoft_phi2",
               MLModelFlavours.pytorch,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )